In [ ]:
!pip install pyspark

## Task 2.1

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Task2").getOrCreate()

purchases_df = spark.read.csv("transactions.csv", header=True, inferSchema=True)
purchases_df.createOrReplaceTempView("PurchasesTable")

query = """
    SELECT *
    FROM PurchasesTable
    WHERE TransTotal <= 600
"""

filtered_purchases_df = spark.sql(query)

filtered_purchases_df.show()

spark.stop()

+-------+------+----------+-------------+--------------------+
|TransID|CustID|TransTotal|TransNumItems|           TransDesc|
+-------+------+----------+-------------+--------------------+
|      9| 48266|    566.61|            2|sVVwWicTRmdCBUcOo...|
|     13| 35742|    304.45|           15|xKFvpjhdoUnAdyLIO...|
|     16| 27652|     17.72|            4|qZGqmCCXHJonYvQym...|
|     18|  1953|    355.19|           11|BeBxWmIJhWoyVrtXW...|
|     19|  6141|    196.05|           15|cqgcAazCfjDQdUFrN...|
|     21| 15248|     88.59|           11|ilaMeBpUKdYANtcSI...|
|     25| 36782|    148.57|            9|CnWqahdwiCAoKSHrX...|
|     29| 45963|     237.1|            2|NbEqOCtERMsKPOSsX...|
|     31| 27494|    353.71|           10|xijqTLuzfNLsxsclx...|
|     36|   426|    377.13|            6|XvlczPJkWeufSWdwr...|
|     37| 49730|    355.75|           10|lxaHGTUAJjQThpoyZ...|
|     39| 45754|    218.12|            3|VswpfdFJukOdRRHUD...|
|     49|  6339|      30.3|            4|vcJJdKLNvPeTRI

## Task 2.2

In [ ]:
from pyspark.sql.functions import expr

spark = SparkSession.builder.appName("Task2").getOrCreate()

purchases_df = spark.read.csv("transactions.csv", header=True, inferSchema=True)

purchases_df.createOrReplaceTempView("PurchasesTable")

query = """
    SELECT *
    FROM PurchasesTable
    WHERE TransTotal <= 600
"""

filtered_purchases_df = spark.sql(query)
filtered_purchases_df.createOrReplaceTempView("FilteredPurchasesTable")

query = """
    SELECT
        TransNumItems,
        percentile_approx(TransTotal, 0.5) AS Median,
        MIN(TransTotal) AS MinTransTotal,
        MAX(TransTotal) AS MaxTransTotal
    FROM (
        SELECT
            TransNumItems,
            SUM(TransTotal) AS TransTotal
        FROM FilteredPurchasesTable
        GROUP BY TransNumItems
    )
    GROUP BY TransNumItems
"""

result_df = spark.sql(query)

result_df.show()

spark.stop()

+-------------+--------------------+--------------------+--------------------+
|TransNumItems|              Median|       MinTransTotal|       MaxTransTotal|
+-------------+--------------------+--------------------+--------------------+
|           12|3.0229836060000014E7|3.0229836060000014E7|3.0229836060000014E7|
|            1|3.0174859800000016E7|3.0174859800000016E7|3.0174859800000016E7|
|           13|       3.017066666E7|       3.017066666E7|       3.017066666E7|
|            6|3.0032788969999995E7|3.0032788969999995E7|3.0032788969999995E7|
|            3|3.0117998709999997E7|3.0117998709999997E7|3.0117998709999997E7|
|            5|       2.991866354E7|       2.991866354E7|       2.991866354E7|
|           15|3.0145407620000035E7|3.0145407620000035E7|3.0145407620000035E7|
|            9|3.0179619500000015E7|3.0179619500000015E7|3.0179619500000015E7|
|            4|       3.010834049E7|       3.010834049E7|       3.010834049E7|
|            8|3.0081506270000014E7|3.00815062700000

## Task 2.3

In [ ]:
spark = SparkSession.builder.appName("Task3").getOrCreate()

purchases_df = spark.read.csv("transactions.csv", header=True, inferSchema=True)
customer_df = spark.read.csv("customer.csv", header=True, inferSchema=True)

purchases_df.createOrReplaceTempView("PurchasesTable")

query = """
    SELECT *
    FROM PurchasesTable
    WHERE TransTotal <= 600
"""

filtered_purchases_df = spark.sql(query)

joined_df = filtered_purchases_df.join(customer_df, filtered_purchases_df["CustID"] == customer_df["ID"])
joined_df.createOrReplaceTempView("JoinedFilteredPurchasesTable")

query = """
    SELECT
        CustID AS CustomerID,  -- Renaming the column to match the expected output
        Age,
        COUNT(TransNumItems) AS TotalNumItems,
        SUM(TransTotal) AS TotalAmountSpent
    FROM JoinedFilteredPurchasesTable
    WHERE Age >= 18 AND Age <= 25
    GROUP BY CustID, Age
"""

result_df = spark.sql(query)

result_df.show()

spark.stop()

+----------+---+-------------+------------------+
|CustomerID|Age|TotalNumItems|  TotalAmountSpent|
+----------+---+-------------+------------------+
|     31946| 23|           39|12777.939999999999|
|     26110| 25|           36|          11080.85|
|     19854| 25|           28|           8507.85|
|     17105| 20|           29| 7427.209999999997|
|     17699| 20|           26| 7851.299999999999|
|     21927| 21|           30|           9275.76|
|      4183| 18|           19|           6316.32|
|     27229| 18|           21|           6356.02|
|     17596| 19|           27|           8838.59|
|      6463| 20|           45|          14360.08|
|      1863| 24|           27|10064.380000000001|
|      9062| 19|           36| 9976.779999999999|
|     12524| 24|           23|7513.2699999999995|
|     29620| 21|           29|           7839.07|
|     14710| 24|           31| 8896.519999999999|
|     42081| 22|           41|          13340.56|
|     44916| 19|           33| 8909.920000000002|


## Task 2.4

In [ ]:
spark = SparkSession.builder.appName("Task4").getOrCreate()

purchases_df = spark.read.csv("transactions.csv", header=True, inferSchema=True)
customer_df = spark.read.csv("customer.csv", header=True, inferSchema=True)

purchases_df.createOrReplaceTempView("PurchasesTable")

query = """
    SELECT *
    FROM PurchasesTable
    WHERE TransTotal <= 600
"""

filtered_purchases_df = spark.sql(query)

joined_df = filtered_purchases_df.join(customer_df, filtered_purchases_df["CustID"] == customer_df["ID"])

joined_df.createOrReplaceTempView("JoinedFilteredPurchasesTable")

query = """
    SELECT
        CustID AS CustomerID,  -- Renaming the column to match the expected output
        Age,
        COUNT(TransNumItems) AS TotalNumItems,
        SUM(TransTotal) AS TotalAmountSpent
    FROM JoinedFilteredPurchasesTable
    WHERE Age >= 18 AND Age <= 25
    GROUP BY CustID, Age
"""

result_df = spark.sql(query)

result_df.createOrReplaceTempView("T3Table")

query = """
    SELECT
        t1.CustomerID AS C1_ID,
        t2.CustomerID AS C2_ID,
        t1.Age AS Age1,
        t2.Age AS Age2,
        t1.TotalAmountSpent AS TotalAmount1,
        t2.TotalAmountSpent AS TotalAmount2,
        t1.TotalNumItems AS TotalItemCount1,
        t2.TotalNumItems AS TotalItemCount2
    FROM T3Table t1
    JOIN T3Table t2 ON t1.Age < t2.Age AND t1.TotalAmountSpent > t2.TotalAmountSpent AND t1.TotalNumItems < t2.TotalNumItems
"""

result_t4_df = spark.sql(query)

result_t4_df.show()

spark.stop()

+-----+-----+----+----+------------------+------------------+---------------+---------------+
|C1_ID|C2_ID|Age1|Age2|      TotalAmount1|      TotalAmount2|TotalItemCount1|TotalItemCount2|
+-----+-----+----+----+------------------+------------------+---------------+---------------+
|31946| 5411|  23|  24|12777.939999999999|11607.859999999999|             39|             42|
|31946| 7350|  23|  25|12777.939999999999|11742.789999999999|             39|             41|
|31946|22154|  23|  25|12777.939999999999|          11750.67|             39|             40|
|31946|33473|  23|  24|12777.939999999999|11270.860000000002|             39|             40|
|31946| 3964|  23|  25|12777.939999999999|11716.500000000002|             39|             40|
|31946|34640|  23|  24|12777.939999999999|12006.230000000001|             39|             40|
|31946|19792|  23|  25|12777.939999999999|            9896.6|             39|             40|
|31946|18875|  23|  24|12777.939999999999|12714.660000000003